In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings

# 忽略警告，让输出干净点
warnings.filterwarnings('ignore')

# 设置中文字体 (为了图表能显示中文)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# ==========================================
# 🔮 模块一：Task 1 趋势感知预测器 (Dual-Fitting Predictor)
# ==========================================
class DemandPredictor:
    def __init__(self, raw_data):
        self.raw_data = raw_data
        self.history_model = None
        self.realtime_lookup = {}
        
        # 1. 训练宏观模型 (30天大规律)
        self._train_history_model()
        
        # 2. 准备微观数据 (用于实时拟合)
        self._prepare_realtime_data()

    def _prepare_realtime_data(self):
        """预处理：把每一分钟的真实数据存好，方便快速查表"""
        temp = self.raw_data.copy()
        temp['Time'] = pd.to_datetime(temp['Time'])
        # 算出绝对分钟数 (0~1440)
        temp['Minute_Index'] = temp['Time'].dt.hour * 60 + temp['Time'].dt.minute
        # 存入字典：{分钟: 人数}
        self.realtime_lookup = temp.groupby('Minute_Index').size().to_dict()

    def _train_history_model(self):
        """宏观：30天数据叠加，拟合6次多项式"""
        print("🔮 [Task 1] 正在训练：宏观趋势(6次) + 微观趋势(1次)...")
        traffic = self.raw_data.set_index('Time').resample('5min').size().reset_index()
        traffic.columns = ['Time', 'Count']
        traffic['minutes'] = traffic['Time'].dt.hour * 60 + traffic['Time'].dt.minute
        
        x = traffic['minutes'].values
        y = traffic['Count'].values
        
        if len(x) > 10:
            # 宏观用高次拟合，捕捉双峰结构
            coeffs = np.polyfit(x, y, deg=6)
            self.history_model = np.poly1d(coeffs)
        else:
            # 数据太少时的兜底逻辑
            self.history_model = lambda m: 0

    def get_realtime_trend_data(self, current_seconds, window=15):
        """
        🌟 核心接口：同时返回 '预测值' 和 '增长斜率'
        """
        current_minutes = (current_seconds // 60) % 1440
        x_local = []
        y_local = []
        
        # 收集过去 15 分钟的真实数据点 (步长5分钟)
        for m in range(current_minutes - window, current_minutes, 5):
            val = self.realtime_lookup.get(m % 1440, 0)
            x_local.append(m)
            y_local.append(val)
            
        # 安全检查：如果数据点太少（刚开局），返回0
        if len(x_local) < 2:
            return 0, 0 

        # 微观：1次拟合 (直线 y = kx + b)
        coeffs = np.polyfit(x_local, y_local, deg=1)
        slope = coeffs[0] # 斜率 k
        
        # 结合宏观和微观计算预测值
        func = np.poly1d(coeffs)
        pred_realtime = max(0, func(current_minutes))
        pred_history = max(0, self.history_model(current_minutes))
        
        # 融合预测 (0.6 历史 + 0.4 实时)
        final_pred = 0.6 * pred_history + 0.4 * pred_realtime
        
        return final_pred, slope

# ==========================================
# 🧠 模块二：Task 2 智能模式选择器 (Mode Selector)
# ==========================================
class ModeSelector:
    def __init__(self, raw_data):
        self.raw_data = raw_data
        self.valid_windows = {
            'UP_PEAK': (6, 10), 'LUNCH': (11, 14), 'DOWN_PEAK': (16, 20)
        }
        self.thresholds = {}
        self._learn_activation_thresholds()

    def _learn_activation_thresholds(self):
        print("🧠 [Task 2] 正在建立双轨触发机制...")
        temp = self.raw_data.copy()
        temp['Time'] = pd.to_datetime(temp['Time'])
        traffic = temp.set_index('Time').resample('5min').size()
        
        if len(traffic) > 0:
            # 📉 降低阈值，配合趋势判断
            base_line = traffic.quantile(0.55)
            alert_line = traffic.quantile(0.40)
        else:
            base_line = 10
            alert_line = 5
            
        self.thresholds['HARD_TRIGGER'] = max(8, base_line)   # 硬触发线
        self.thresholds['SOFT_TRIGGER'] = max(5, alert_line)  # 软触发线
        self.thresholds['TREND_SLOPE'] = 1.5                  # 爆发斜率阈值
        
        print(f"✅ 阈值设定：硬线={self.thresholds['HARD_TRIGGER']:.1f}, 软线={self.thresholds['SOFT_TRIGGER']:.1f}, 爆发斜率={self.thresholds['TREND_SLOPE']}")

    def get_mode(self, current_seconds, pred_demand, trend_slope):
        """
        🔥 升级版判断逻辑：结合 绝对值 + 趋势
        """
        hour = (current_seconds // 3600) % 24
        
        # 1. 时间窗筛选
        possible_mode = 'OFF_PEAK'
        for mode, (start, end) in self.valid_windows.items():
            if start <= hour < end:
                possible_mode = mode
                break
        
        if possible_mode == 'OFF_PEAK': return 'OFF_PEAK'

        # 2. 🔥 双轨判断 (Dual-Track Trigger)
        # 轨道A: 人多 (硬触发)
        if pred_demand >= self.thresholds['HARD_TRIGGER']:
            return possible_mode
            
        # 轨道B: 人虽少但涨得快 (软触发 + 趋势补偿)
        elif (pred_demand >= self.thresholds['SOFT_TRIGGER']) and (trend_slope >= self.thresholds['TREND_SLOPE']):
            # print(f"⚠️ [趋势触发] {hour}点检测到激增 (k={trend_slope:.2f})") # 调试用
            return possible_mode
            
        return 'OFF_PEAK'

# ==========================================
# ⚙️ 模块三：Task 3 动态群控策略 (Group Control)
# ==========================================

# [距离, 方向, 载重, 任务数, 势能, 拥挤排斥]
MODE_WEIGHTS = {
    'UP_PEAK':   [1.0, 15.0, 1.0, 1.0, 20.0, 10.0], # 早高峰：势能(w5)最大
    'DOWN_PEAK': [1.0, 10.0, 1.0, 2.0, 5.0,  30.0], # 晚高峰：排斥(w6)最大
    'LUNCH':     [2.0, 2.0,  0.5, 0.5, 2.0,  5.0],  
    'OFF_PEAK':  [1.0, 1.0,  10.0, 1.0, 0.0, 0.0]   
}

def calculate_dynamic_cost(elev, req_floor, mode, pred_demand, slope, all_elevators):
    """
    🔥 终极代价函数：包含 趋势放大 + 空间排斥
    """
    w1, w2, w3, w4, w5, w6 = MODE_WEIGHTS[mode]
    
    # 1. 基础物理代价
    dist_cost = abs(elev.current_floor - req_floor)
    load_cost = elev.load_count
    task_cost = len(elev.stops)
    
    # 2. 方向惩罚
    dir_cost = 0
    if elev.state != 'IDLE':
        if (elev.direction == 1 and req_floor < elev.current_floor) or \
           (elev.direction == -1 and req_floor > elev.current_floor):
            dir_cost = 20 
            
    # 3. 🌟 趋势增强型势能场 (Trend-Amplified Potential)
    potential_cost = 0
    # 油门系数：如果人流疯涨 (slope>0)，势能项加倍
    trend_factor = 1.0 + max(0, slope * 0.5) 
    
    if mode == 'UP_PEAK':
        potential_cost = (elev.current_floor * pred_demand) * trend_factor
    elif mode == 'DOWN_PEAK':
        potential_cost = ((30 - elev.current_floor) * (pred_demand * 0.1)) * trend_factor

    # 4. 🤝 空间排斥 (Spatial Repulsion)
    bunching_cost = 0
    if mode != 'OFF_PEAK':
        for teammate in all_elevators:
            if teammate.id == elev.id: continue
            
            # 如果队友离我很近 (<3层) 且方向相同 -> 罚分
            dist_to_mate = abs(teammate.current_floor - elev.current_floor)
            if dist_to_mate < 3 and teammate.direction == elev.direction:
                bunching_cost += (10.0 / (dist_to_mate + 1))

    # 5. 总分
    total_cost = (w1 * dist_cost) + (w2 * dir_cost) + (w3 * load_cost) + \
                 (w4 * task_cost) + (w5 * potential_cost) + (w6 * bunching_cost)
                 
    return total_cost

# ==========================================
# 🏗️ 仿真基础对象
# ==========================================
class SmartElevator:
    def __init__(self, id):
        self.id = id
        self.current_floor = 1
        self.direction = 0 
        self.state = 'IDLE'
        self.stops = set() 
        self.passengers = [] 
        self.capacity = 20
        # 统计指标
        self.total_trips = 0
    
    @property
    def load_count(self):
        return len(self.passengers)

    def move(self):
        if self.state == 'IDLE': return
        
        self.current_floor += self.direction
        
        if self.current_floor in self.stops:
            self.stops.remove(self.current_floor)
            self.passengers = [p for p in self.passengers if p['dest'] != self.current_floor]
            self.total_trips += 1 # 简单统计停靠次数
            
        if not self.stops:
            self.state = 'IDLE'
            self.direction = 0
        elif max(self.stops, default=0) > self.current_floor:
            self.direction = 1
        elif min(self.stops, default=99) < self.current_floor:
            self.direction = -1

# ==========================================
# 🎮 仿真总控中心 (支持随机抽取某天)
# ==========================================
class SimulationController:
    def __init__(self, data_path):
        try:
            self.full_df = pd.read_csv(data_path)
        except:
            self.full_df = pd.read_csv(data_path, encoding='gbk')
            
        self.full_df['Time'] = pd.to_datetime(self.full_df['Time'])
        self.predictor = DemandPredictor(self.full_df)
        
    def run_random_day_simulation(self):
        # 1. 🎲 随机抽取一天
        unique_dates = self.full_df['Time'].dt.date.unique()
        if len(unique_dates) == 0:
            print("数据为空！")
            return
            
        selected_date = random.choice(unique_dates)
        print(f"🎲 随机抽中日期: {selected_date}")
        print(f"📅 正在加载该日数据并进行全天模拟 (24小时)...")
        
        # 2. 过滤出这一天的数据 (作为 Ground Truth)
        day_df = self.full_df[self.full_df['Time'].dt.date == selected_date].copy()
        
        # 准备"当天专用"的实时查询表 {分钟: 人数}
        # 这一步是为了模拟：在仿真过程中，Task1只能看到"这一天"过去发生的事
        day_df['Minute'] = day_df['Time'].dt.hour * 60 + day_df['Time'].dt.minute
        day_lookup = day_df.set_index('Time').resample('5min').size()
        # 把 resample 后的 index 转成 0-1440 的分钟整数，做成字典方便查询
        day_lookup_dict = { (t.hour * 60 + t.minute): count for t, count in day_lookup.items() }
        
        # 3. 准备绘图数据容器
        time_axis = []      # X轴：时间
        actual_curve = []   # Y轴1：真实值
        pred_curve = []     # Y轴2：预测值
        
        # 4. ⏱️ 模拟 24 小时 (每 5 分钟为一个步长，加速模拟)
        # 我们这里不按秒跑，为了画图快一点，按5分钟跑一次预测
        for t_min in range(0, 1440, 5): 
            t_seconds = t_min * 60
            
            # --- A. 获取预测值 ---
            # 注意：把"当天"的数据表传进去，模拟实时获取了当天的数据
            pred_val, slope = self.predictor.get_realtime_trend_data(t_seconds, day_lookup_dict)
            
            # --- B. 获取真实值 (Ground Truth) ---
            actual_val = day_lookup_dict.get(t_min, 0)
            
            # --- C. 记录数据 ---
            time_axis.append(t_min / 60.0) # 换算成小时
            actual_curve.append(actual_val)
            pred_curve.append(pred_val)

        # 5. 📊 画图对比 (Visualization)
        self._plot_results(selected_date, time_axis, actual_curve, pred_curve)
        
    def _plot_results(self, date, x, y_actual, y_pred):
        plt.figure(figsize=(12, 6))
        
        # 画真实值 (虚线 + 点)
        plt.plot(x, y_actual, label='真实流量 (Ground Truth)', color='#1f77b4', linestyle='--', alpha=0.6, linewidth=1.5)
        
        # 画预测值 (实线 - 红色)
        plt.plot(x, y_pred, label='模型预测 (Predicted)', color='#d62728', linewidth=2.5)
        
        # 装饰图表
        plt.title(f'Task 1 预测性能验证: {date} (随机抽取)', fontsize=16)
        plt.xlabel('时间 (小时)', fontsize=12)
        plt.ylabel('5分钟人流量', fontsize=12)
        plt.xticks(range(0, 25, 2)) # X轴刻度：每2小时一个
        plt.grid(True, linestyle=':', alpha=0.6)
        plt.legend(fontsize=12)
        
        # 填充颜色，显示预测偏差
        plt.fill_between(x, y_actual, y_pred, color='gray', alpha=0.1, label='预测误差')
        
        plt.tight_layout()
        plt.show()
        print("✅ 图表生成完毕！红色实线是你的算法，蓝色虚线是那天实际发生的事。")

# ==========================================
# ▶️ 执行区域
# ==========================================
if __name__ == "__main__":
    # 请修改为你的文件路径
    data_path = r'MCM2026/mcm26Train-B-Data/hall_calls.csv'
    
    try:
        # 初始化
        sim = SimulationController(data_path)
        # 🔥 随机抽取一天并画图
        sim.run_random_day_simulation()
        
    except FileNotFoundError:
        print("❌ 找不到文件，请上传 hall_calls.csv 或修改 data_path 路径")
    except Exception as e:
        print(f"❌ 运行出错: {e}")
        # 如果是中文字体报错，可以尝试把上面的 plt.rcParams 代码注释掉
    # ⚠️ 请确保这里的路径是你电脑上的真实路径
    # 比如 r'MCM2026/mcm26Train-B-Data/hall_calls.csv'
    data_path = 'hall_calls.csv' 
    
    # 为了演示方便，如果找不到文件，请手动修改上面的 data_path
    try:
        sim = SimulationController(data_path)
        # 跑 2 小时 (7200秒)
        sim.run(duration=7200)
    except FileNotFoundError:
        print("❌ 错误：找不到文件。请修改代码最后的 data_path 变量！")

❌ 找不到文件，请上传 hall_calls.csv 或修改 data_path 路径
❌ 错误：找不到文件。请修改代码最后的 data_path 变量！
